<a href="https://colab.research.google.com/github/AleCongi/Esame-ICON-/blob/main/CNN_for_event_posters_and_OCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **I want to create a convolutional neural network that takes 2 types of images:poster of events , and images which are not poster of events ,  determines if  the image is indeed a poster and for those above a set level of  certanty then it   extracts the text inside it with OCR which will be later processed by other modules for NLP that will be able through mulitple source of knowledge aquired  predicts these features  reguarding the Event like :the name,the time and date, the place,the organizer,the type and general vibe,the latitude and longitude,the involvment of other entities like organizers or individuals,possibly the availability of tickets and tokens with the multiple ways to acquire them,the estimated number of partecipants and so on.. .**

 I will use  a pre-trained model:VGG which is a model that has already been trained on a large dataset ,Imagenet in this case, and can be used as a starting point for solving a similar but different task. The idea behind using pre-trained models is that they have already learned useful features from the data they were trained on, which can be fine-tuned to the new task using a smaller dataset.

In [1]:
import keras
import matplotlib.pyplot as plt
from keras  import datasets,layers,models
from keras.applications import VGG16
from keras import losses
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Reshape

I will advise you to use Google Drive or other alike serices to load the immages since colab has an annoying refresh rate every couple of hours which deletes every file present on your folder

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
event_path = "/content/drive/MyDrive/Colab Notebooks/yesPosters"
non_event_path = "/content/drive/MyDrive/Colab Notebooks/noPosters"

Mounted at /content/drive


I label each image with the correct class (event poster or non-event poster). This will allow the model to learn from the labeled data.

I made sure in the Crawling phase  to balance the number of images for each class to feed this model, if there's a large imbalance in the dataset, the model might perform poorly.

The image size is resized to (224, 224) pixels because it is a common size that is used in many pre-trained models such as VGG, ResNet, and Inception. These models were trained on a large dataset of images, and the images were resized to this size before being fed into the model.

The reason why the image size of (224, 224) is so widely used in pre-trained models is because it is a good balance between the ability to capture fine-grained details in an image and computational efficiency.
A larger image size would require more computation power and memory, which can be a bottleneck in some cases.

Another reason is that many datasets used to train these models were collected and labeled with image size of (224, 224) pixels, so models trained on those datasets are optimized to work well with images of that size.
In addition, this size is also compatible with most of the available hardware and software, which makes it more accessible for a wide range of users.

In [3]:
# Initialize empty lists to store the images and labels
images = []
labels = []

# Loop through the event posters and add them to the images and labels list
for filename in os.listdir(event_path):
    img = cv2.imread(os.path.join(event_path, filename))
    img = cv2.resize(img, (224, 224))
    images.append(img)
    labels.append(1)

# Loop through the non-event posters and add them to the images and labels list
for filename in os.listdir(non_event_path):
    img = cv2.imread(os.path.join(non_event_path, filename))
    img = cv2.resize(img, (224, 224))
    images.append(img)
    labels.append(0)


An image is a 2-dimensional matrix of pixels. Each pixel in an image has a value that represents its color. The value of each pixel is usually represented by 3 values (R, G, B) which stands for Red, Green and Blue color channels respectively.

When an image is loaded into python, it is usually represented as a numpy array, where each element of the array represents the value of a pixel. Each element of the array is a tuple of 3 values (R, G, B) which represent the color of the pixel.

For example, an image of size (224, 224, 3) would be represented as a numpy array of shape (224, 224, 3) with each element of the array representing a pixel. The first two dimensions of the array represent the height and width of the image and the last dimension represents the color channel.

So when using images = np.array(images), it converts the list of images, where each image is represented as a 2-dimensional matrix of pixels, into a 3-dimensional numpy array where each element of the array represents a pixel in an image.

I'm using jpeg with RGB color space which  has 3 channels: Red, Green and Blue.

The "bits per color channel" (bpc) is the number of bits that are used for storing each component 24 bit in my case , therefore each channel ranges from 0 to 256 so the last dim of the array for each three coordinates will go from zero to 256 max.

It's worth noting that some image formats like grayscale images, have only one channel, in this case, the shape of the numpy array will be (224, 224) instead of (224, 224, 3)

In [7]:
# Convert the lists to numpy arrays
images = np.array(images)
labels = np.array(labels)
print(images.shape)
print(labels.shape)


# Split the data into training and test sets
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42,stratify=labels)

(2036, 224, 224, 3)
(2036,)


In [9]:
print(x_train.shape)
print(y_train.shape)

print(x_test.shape)
print(y_test.shape)

(1628, 224, 224, 3)
(1628,)
(408, 224, 224, 3)
(408,)


In [81]:
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

x_train shape: (1628, 224, 224, 3)
1628 train samples
408 test samples
x_train shape: (1628, 224, 224, 3)
1628 train samples
408 test samples
y_train shape: (1628,)
y_test shape: (408,)


In [11]:
from keras.applications import VGG16
model = VGG16(weights='imagenet', include_top=False,input_shape=(224, 224, 3))
from keras.models import Model



58889256/58889256 [==============================] - 0s 0us/step


In [13]:

batch_size = 128
num_classes = 2
epochs = 12


In [84]:
print(x_train[0])

[[[ 24  91 154]
  [ 21  97 159]
  [ 27 107 168]
  ...
  [ 32  61 146]
  [ 30  59 144]
  [ 24  53 138]]

 [[ 37 104 167]
  [ 36 112 174]
  [ 31 111 172]
  ...
  [ 35  68 152]
  [ 35  67 152]
  [ 28  60 145]]

 [[ 33 106 164]
  [ 27 105 162]
  [ 32 113 170]
  ...
  [ 23  59 147]
  [ 27  61 149]
  [ 34  66 152]]

 ...

 [[ 26  22  98]
  [ 26  22  98]
  [ 26  22  98]
  ...
  [ 26  22  98]
  [ 26  22  98]
  [ 26  22  98]]

 [[ 26  22  98]
  [ 26  22  98]
  [ 26  22  98]
  ...
  [ 26  22  98]
  [ 26  22  98]
  [ 26  22  98]]

 [[ 26  22  98]
  [ 26  22  98]
  [ 26  22  98]
  ...
  [ 26  22  98]
  [ 26  22  98]
  [ 26  22  98]]]


Freezing the layers of a pre-trained model, except the final fully connected layer, is done to prevent the model from forgetting the features it has already learned. The pre-trained model has already learned useful features from the data it was trained on, and it's important to keep these learned features while fine-tuning the model on a new dataset.
The VGG16 model that you used is pre-trained on the ImageNet dataset and is designed to recognize 1000 different classes of objects. The final layer of the model is a fully connected layer with 1000 units, each corresponding to one of the 1000 classes.

In my task, i want to use the VGG16 model to predict if an image is an event poster or not, which is a binary classification task. So i need to replace the final fully connected layer of the VGG16 model with a new fully connected layer that has only 2 units, one for each class (event poster and non-event poster).

I removed the last fully connected layer by using model.layers.pop() and then added a new fully connected layer with only 2 units using the Dense(1, activation='sigmoid') layer. This new layer is connected to the output of the VGG16 model, and its output will be used as the final prediction for the task.

I also added in between a Flatten level since the fully connected layers in a neural network expect their input to be a one-dimensional array, so the Flatten() layer is used to convert the 3D tensor( The output of the VGG16 model is a 3D tensor of shape (batch_size, height, width, channels)) into a 1D array, which can then be used as input to the fully connected layers.

I added finally a new dense layer with 256 units and relu activation function, this dense layer is added to increase the representational power of the model,it's called bottleneck layer. It's used to prevent overfitting and increase the ability of the model to generalize.

In [17]:
# Remove the last fully connected layer
model.layers.pop()

# Freeze all the layers
for layer in model.layers:
    layer.trainable = False

# Add a new fully connected layer
x = model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)

# Compile the model
model = Model(inputs=model.input, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
print(x_train)
print(y_train)
y_train = np.asarray(y_train).astype('float32').reshape((-1,1))
y_test = np.asarray(y_test).astype('float32').reshape((-1,1))
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))


[[[[ 64  64  64]
   [ 64  64  64]
   [ 64  64  64]
   ...
   [ 64  64  64]
   [ 64  64  64]
   [ 64  64  64]]

  [[ 64  64  64]
   [ 64  64  64]
   [ 64  64  64]
   ...
   [ 64  64  64]
   [ 64  64  64]
   [ 64  64  64]]

  [[ 64  64  64]
   [ 64  64  64]
   [ 64  64  64]
   ...
   [ 64  64  64]
   [ 64  64  64]
   [ 64  64  64]]

  ...

  [[ 63  63  63]
   [ 63  63  63]
   [ 61  61  61]
   ...
   [ 63  63  63]
   [ 63  63  63]
   [ 63  63  63]]

  [[ 64  64  64]
   [ 64  64  64]
   [ 63  63  63]
   ...
   [ 64  64  64]
   [ 64  64  64]
   [ 64  64  64]]

  [[ 64  64  64]
   [ 64  64  64]
   [ 64  64  64]
   ...
   [ 64  64  64]
   [ 64  64  64]
   [ 64  64  64]]]


 [[[  1   1   1]
   [  1   1   1]
   [  1   1   1]
   ...
   [  1   1   1]
   [  1   1   1]
   [  1   1   1]]

  [[  1   1   1]
   [  1   1   1]
   [  1   1   1]
   ...
   [  1   1   1]
   [  1   1   1]
   [  1   1   1]]

  [[  1   1   1]
   [  1   1   1]
   [  1   1   1]
   ...
   [  1   1   1]
   [  1   1   1]
   [  1   1

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Training a model can take a significant amount of time, especially when working with large datasets or complex models. Saving the trained model allows you to use it later without having to retrain it.
You can save the architecture of the model using the model.to_json() method, and save the weights using the model.save_weights() method. Later, you can load the architecture and weights separately using the model_from_json() and model.load_weights() methods.
You can save the entire model like im doing, including the architecture, weights, and optimizer state, using the model.save() method. Later, you can load the entire model using the keras.models.load_model() method.

In [ ]:
model.save('eventPosterRecognizer.h5')

# Now i will discuss the Optical Character Recognition(OCR) part, which i will be  using to extract as much semantic as possible from the image.

Python-tesseract is a wrapper for Google’s Tesseract-OCR Engine. It is also useful as a stand-alone invocation script to tesseract, as it can read all image types supported by the Pillow and Leptonica imaging libraries, including jpeg, png, gif, bmp, tiff, and others. Additionally, if used as a script, Python-tesseract will print the recognized text instead of writing it to a file.

In [16]:
!pip install pytesseract

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
from PIL import Image
from pytesseract import pytesseract
import shutil
import os
import cv2
import json
import random
try:
 from PIL import Image
except ImportError:
 import Image

By the way you will need to also have the binary associated with it ,download it from the site and remember the PATH where you have stored it.

In [15]:
!sudo apt install tesseract-ocr

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 21 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 2s (2,695 kB/s)
debconf: unable to initi

Let's load the model we have previously saved and the images
and resize the image, my model was trained on 224x224X3 pictures so before making predictions you need to convert these images too.

In [ ]:
loaded_model = keras.models.load_model('eventPosterRecognizer.h5')
new_images= "/content/drive/MyDrive/Colab Notebooks/newImages"
# Resize the new images to (224, 224)
new_images = [cv2.resize(image, (224, 224)) for image in new_images]

The predict() function will return an array of probabilities for each class, I will  choose a threshold to decide if the image is an event poster or not and save the result in a json file with a parameter for the image path.

In [ ]:
predictions = loaded_model.predict(new_images)
treshold=0.8
# Initialize a list to store the extracted text and image information
text_list = []

the image is first converted to grayscale using the cv2.cvtColor() function. Then, Otsu thresholding is applied to the grayscale image using the cv2.threshold() function to improve the quality of the text recognition. Finally, the image is passed to the pytesseract.image_to_string() function with the lang='ita+eng' parameter to specify that the text is in Italian or English. This will return the recognized text as a string.

In [ ]:
def extract_text(image):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Otsu thresholding
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Pass the image to tesseract
    text = pytesseract.image_to_string(thresh, lang='ita+eng')

    return text

In [ ]:
# Loop through the predictions and extract text from the images with a prediction greater than the threshold
for i, image in enumerate(new_images):
    if predictions[i][1] > threshold:
        # Extract text from the image
        text = extract_text(image)
        # Append the text and image information to the list
        text_list.append({"text": text, "image_path": image_paths[i]})

# Save the list as a json file
with open('text_data.json', 'w') as f:
    json.dump(text_list, f)
        
